In [1]:
#%pip install transformers==4.17.0
#!git clone https://github.com/nguyenvulebinh/extractive-qa-mrc
%cd extractive-qa-mrc

D:\_anaconda\extractive-qa-mrc


In [2]:
from pyvi.ViTokenizer import tokenize
import re, os, string
import pandas as pd
import math
import numpy as np

In [3]:
from infer import tokenize_function, data_collator, extract_answer
from model.mrc_model import MRCQuestionAnswering
from transformers import AutoTokenizer
import nltk
nltk.download('punkt')

D:\anaconda3\envs\gpu_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Xây dựng BM25
### Đầu vào sẽ là list[list[term]], với list[term] là list các term của 1 document
### k1 là trọng số cho tần suất từ (TF).
### b là trọng số kiểm soát ảnh hưởng của độ dài tài liệu đến score. Với b gần 1, tài liệu dài sẽ bị trừ điểm; b gần 0 thì ngược lại.

### tf: list[dict[str,int]] - số lần xuất hiện của từ trong doc.
### df: dict[str, int] - số doc chứa term trong tập doc.
### idf: dict[str, float] - IDF của term
### doc_len_: list[int] - số term trong mỗi doc
### corpus_: list[list[str]]
### corpus_size_: int - Số lượng doc trong tập doc
### avg_doc_len_: float - độ dài trung bình của doc


In [4]:
class BM25:
    def __init__(self, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1

    def fit(self, corpus):
        tf = []
        df = {}
        idf = {}
        doc_len = []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count

            tf.append(frequencies)

            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        for term, freq in df.items():
            idf[term] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))

        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self

    def search(self, query):
        scores = [self._score(query, index) for index in range(self.corpus_size_)]
        return scores

    def _score(self, query, index):
        score = 0.0

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score += (numerator / denominator)

        return score

## Các bước tiền xử lí đưa document thành list các term

In [5]:
def clean_text(text):
    text = re.sub('<.*?>', '', text).strip() # xóa tag của html
    text = re.sub('(\s)+', r'\1', text) # đưa nhiều dấu cách/tab/xuống dòng thành 1 dấu cách/tab/xuống dòng
    return text

In [6]:
def normalize_text(text): #Loại bỏ các dấu câu trừ gạch dưới và chuyển thành chữ thường
    listpunctuation = string.punctuation.replace('_', '')
    for i in listpunctuation:
        text = text.replace(i, ' ')
    return text.lower()

In [7]:
normalize_text('Đây.là &ví dụ')

'đây là  ví dụ'

In [8]:
clean_text("<tag>    Đây là    ví dụ  ")

'Đây là ví dụ'

In [9]:
def word_segment(sent): #token hóa câu
    sent = tokenize(sent.encode('utf-8').decode('utf-8'))
    return sent

In [10]:
word_segment('Đây là ví dụ')

'Đây là ví_dụ'

In [11]:
filename = 'stopwords.csv'
data = pd.read_csv(filename, sep="\t", encoding='utf-8')
list_stopwords = set(data['stopwords'])

def remove_stopword(text): #Loại bỏ stopword
    pre_text = []
    words = text.split()
    for word in words:
        if word not in list_stopwords:
            pre_text.append(word)
    text2 = ' '.join(pre_text)
    return text2

In [12]:
normalize_text("Đây_là%ví#dụ")

'đây_là ví dụ'

### Gộp tất cả các file trong thư mục lưu trữ dữ liệu thành 1 file

In [13]:
context_dir = 'D:\_anaconda\extractive-qa-mrc\sgk'
file_names = [os.path.join(context_dir, path) for path in os.listdir(context_dir)]

context_all_path = 'D:\_anaconda\extractive-qa-mrc\context.txt'

with open(context_all_path, 'w', encoding='utf8') as combined_file:
    for file_name in file_names:
        with open(file_name, 'r', encoding='utf8') as current_file:
            content = current_file.read()
            combined_file.write(content)
            combined_file.write('\n')

### Tạo list các document

In [14]:
docs = []
doc_ids = []
with open(context_all_path, encoding='utf-8') as f_r:
    contents = f_r.read().strip().split('======================================================================')
    for content in contents:
        doc_id = content.split(' ')[0].strip()
        if doc_id[0:2] != 'c.':
            doc_id = 'blank'
        content = clean_text(content)
        content = word_segment(content)
        content = normalize_text(content)
        content = remove_stopword(content)
        docs.append(content)
        doc_ids.append(doc_id)

### Đưa các document thành list[term]

In [15]:
dictionary = [[word for word in document.lower().split() if word not in list_stopwords] for document in docs]

In [23]:
print(dictionary[224])

['di_tích', 'lịch_sử', 'địa_danh', '…', 'xâm_lược', 'đông', 'nam', 'thực_dân', 'phương', 'tây', 'tồn_tại', 'ngày_nay', 'di_tích', 'lịch_sử', 'địa_danh', 'xâm_lược', 'đông', 'nam', 'thực_dân', 'phương', 'tây', 'tồn_tại', 'ngày_nay', 'tượng_đài_la', 'pu', 'la', 'pu', 'đảo', 'mác', 'tan', 'phi', 'lip', 'pin', 'trường', 'đại_học', 'chu', 'la', 'long', 'kon', 'thái', 'lan']


### Xây dựng hàm tìm kiếm các document(context) phù hợp nhất với câu cần hỏi

In [24]:
def bm25_search(query, limit=3, k1=1.99, b=0.655):
    bm25 = BM25(k1=k1, b=b)
    bm25.fit(dictionary)
    query_processed = clean_text(query)
    query_processed = word_segment(query_processed)
    query_processed = remove_stopword(normalize_text(query_processed))
    query_processed = query_processed.split()

    scores = bm25.search(query_processed)
    scores_index = np.argsort(scores)
    scores_index = scores_index[::-1]
    scores.sort(reverse=True)
    docs_score = scores[:limit]
    context = np.array([contents[i] for i in scores_index])[:limit]
    
    return context, docs_score

## Sử dụng pretrain của nguyenvulebinh

In [25]:
model_checkpoint = "nguyenvulebinh/vi-mrc-large"
# model_checkpoint = "nguyenvulebinh/vi-mrc-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = MRCQuestionAnswering.from_pretrained(model_checkpoint)

### Xây dựng hàm xử lí các đoạn context quá dài bằng cách chia context thành nhiều đoạn context con, mỗi context con sẽ trùng nhau 1 ít

In [26]:
def overlap_context(context, overlap_size, max_size=300):
    context_words = context.split(" ")
    len_context = len(context_words)
    sub_len = max_size - overlap_size
    number_sub = (len_context - overlap_size) // sub_len + 1

    sub_contexts = []
    for i in range(number_sub):
        start = i * sub_len
        end = min(start + max_size, len_context)
        sub_context = context_words[start:end]

        sub_context = " ".join(sub_context)

        sub_contexts.append(sub_context)
    return sub_contexts

### Làm sạch context đưa vào model MRC

In [27]:
def clean_context(text):
    text = text.lower()
    punc = '''![]{}'"\<>/?@#$^&*_~'''

    for ele in text:
        if ele in punc:
            text = text.replace(ele, "")
    return text

### Làm sạch câu trả lời có được từ model MRC

In [28]:
def clean_answer(answer):
    if ' %' in str(answer):
        answer = str(answer.replace(' %', '%'))
    return answer

### Xây dựng hàm trả lời câu hỏi với đầu vào là câu hỏi và context

In [29]:
def get_answer(question, context):
    answers = []
    if len(context.split(" ")) > 295:
        list_contexts = overlap_context(context, 50, max_size=295)
        
        for cont in list_contexts:
            cont = cont.split()
            if 'c.1' in cont[0]:
                cont = ' '.join(cont[1:])
            else:
                cont = ' '.join(cont[0:])
            cont = clean_context(cont)
            QA_input = {
                'question': question,
                'context': cont
            }
            
            inputs = [tokenize_function(QA_input, tokenizer)]
            inputs_ids = data_collator(inputs, tokenizer)
            outputs = model(**inputs_ids)
            answer = extract_answer(inputs, outputs, tokenizer)
            answers.append(answer)
    else:
        context = clean_context(context)
        context = context.split()
        if 'c.1' in context[0]:
            context = ' '.join(context[1:])
        else:
            context = ' '.join(context[0:])
        QA_input = {
                'question': question,
                'context': context
            }
        inputs = [tokenize_function(QA_input, tokenizer)]
        inputs_ids = data_collator(inputs, tokenizer)
        outputs = model(**inputs_ids)
        answers = [extract_answer(inputs, outputs, tokenizer)]
    
    return answers

## Xây dựng hàm thực hiện toàn bộ quá trình: tìm kiếm các document/context tốt nhất dựa trên câu cần hỏi bằng BM25, và đưa các context tốt nhất đó cùng câu cần hỏi vào model MRC

In [30]:
def qa(query, limit=5, k1=1.99, b=0.655):
    contexts = bm25_search(query, limit=limit, k1=k1, b=b)[0]
    for context in contexts:
        results = get_answer(query, context)
    
        for result in results:
            answer = result[0]['answer']
        
            if len(answer) > 0:
                if answer in context:
                    context = context.replace(answer, answer.upper())
                return clean_answer(answer).upper().strip(), context.strip(), [context]+[cont for cont in contexts if cont != context]
    
    return '-1', '-1', list(contexts)

## Kết quả thực nghiệm

### Tạo tập test. Tập test bao gồm list câu hỏi, list câu trả lời đúng và list context chứa câu trả lời đúng

In [31]:
test_path = 'test_question.txt'

with open(test_path, 'r', encoding = 'utf-8') as file:
    raw_test = file.read()
    list_test = raw_test.split('======================================================================')
    list_question = []
    list_answer = []
    list_context = []
    list_doc_id = []

    for i in range(len(list_test)):
        if list_test[i]:
            t = list_test[i].strip().split('\n')
            list_question.append(t[0])
            list_answer.append(t[1])
            list_context.append(t[2])
            list_doc_id.append(t[2].split(' ')[0])

### Xây dựng hàm tính điểm cho BM25. Điểm được tính bằng tổng của 1/j, với j là top của context đúng trong limit context tìm được

#### Ví dụ, ở câu hỏi thứ nhất, BM25 tìm ra 5 context và context chính xác nằm ở context thứ 3 của BM25 thì câu 1 sẽ được 1/3 điểm

In [32]:
def search_accu(test_path, limit=5, k1=1.99, b=0.655):
    file = open(test_path, 'r', encoding='utf8')
    raw_test = file.read()
    list_test = raw_test.split('======================================================================')
    list_question = []
    list_answer = []
    list_context = []
    list_doc_id = []

    for i in range(len(list_test)):
        if list_test[i]:
            t = list_test[i].strip().split('\n')
            list_question.append(t[0])
            list_answer.append(t[1])
            list_context.append(t[2])
            list_doc_id.append(t[2].split(' ')[0])
            
    score = 0
    top_score = [0 for i in range(limit)]
    list_question_error = []
    
    for i in range(len(list_question)):
        result = bm25_search(list_question[i], k1=k1, b=b, limit=limit)[0]
        error = 0
        for j in range(len(result)):
            doc_id_pred = result[j].strip().split()[0]
            if doc_id_pred == list_doc_id[i]:
                error += 1
                top_score[j] += 1
                score += 1/(j+1)
            else:
                score += 0
        if error == 0:
            list_question_error.append(list_question[i])
    
    return score, list_question_error, top_score

In [33]:
def count_err(list_ans):
    count = 0
    for ans in list_ans:
        if ans == '-1':
            count += 1
    return count

In [34]:
from sentence_transformers import SentenceTransformer
from numpy.linalg import norm
import pandas as pd


def cosine(A, B):
    x = np.dot(A, B)
    y = norm(A) * norm(B)
    return x/y

def bm25_search_s(query, limit=3):
    query_processed = clean_text(query)
    query_processed = word_segment(query_processed)
    query_processed = remove_stopword(normalize_text(query_processed))
    query_processed = query_processed.split()

    scores = bm25.search(query_processed)
    scores_index = np.argsort(scores)
    scores_index = scores_index[::-1]

    context = np.array([contents[i] for i in scores_index])[:limit]

    return context

def get_embed(batch_text):
    batch_embedding = simCSE.encode(batch_text)
    return [np.array(vector) for vector in batch_embedding]

def clean_sem(text):
    text = text.lower()
    punc = '''!()[]{};:'\,"”“<>./?@#$%^&*_~'''
    for ele in text:
        if ele in punc:
            text = text.replace(ele, "")
    cleaned_text = ' '.join(text.strip().split())
    return cleaned_text

def reverse_tokenized(tokenized):
    reversed_text = " ".join(tokenized.split("_"))
    reversed_text.replace(" - ", "-")
    reversed_text = reversed_text.replace(" - ", "-")
    return reversed_text

def overlap_splitter(input_string, max_length=256, overlap=20):
    segments = []
    start = 0
    while start < len(input_string):
        end = start + max_length
        segment = input_string[start:end]
        while end < len(input_string) and not input_string[end].isspace():
            end += 1
        segments.append(input_string[start:end].strip())
        start = end - overlap
    return segments

def three_sub_relevant(question, context):
    results = []
    question = clean_sem(question)
    context = clean_sem(context)
    c_tokenized = tokenize(context)
    q_tokenized = tokenize(question)
    chunks = overlap_splitter(c_tokenized)
    q_embed = get_embed(q_tokenized)

    if len(chunks) == 1:
        results.append(context)
        results.append('')
        results.append('')
        return results

    if len(chunks) == 2:
        results.append(reverse_tokenized(chunks[0]))
        results.append(reverse_tokenized(chunks[1]))
        results.append('')
        return results

    embed_chunks = []
    for i in chunks:
        embed_chunks.append(get_embed(i))
    score = [cosine(q_embed, embed_part) for embed_part in embed_chunks]
    top_3_index = np.argsort(score)[::-1][:3]

    results.append(reverse_tokenized(chunks[top_3_index[0]]))
    results.append(reverse_tokenized(chunks[top_3_index[1]]))
    results.append(reverse_tokenized(chunks[top_3_index[2]]))
    return results

def overlap_context(context, overlap_size, max_size=300):
    context_words = context.split(" ")
    len_context = len(context_words)
    sub_len = max_size - overlap_size
    number_sub = (len_context - overlap_size) // sub_len + 1

    sub_contexts = []
    for i in range(number_sub):
        start = i * sub_len
        end = min(start + max_size, len_context)
        sub_context = context_words[start:end]

        sub_context = " ".join(sub_context)

        sub_contexts.append(sub_context)
    return sub_contexts

def get_answer(question, context):
    answers = []
    if len(context.split(" ")) > 300:
        list_contexts = overlap_context(context, 50)

        for cont in list_contexts:
            QA_input = {
                'question': question,
                'context': cont
            }

            inputs = [tokenize_function(QA_input, tokenizer)]
            inputs_ids = data_collator(inputs, tokenizer)
            outputs = model(**inputs_ids)
            answer = extract_answer(inputs, outputs, tokenizer)
            answers.append(answer)
    else:
        QA_input = {
                'question': question,
                'context': context
            }
        inputs = [tokenize_function(QA_input, tokenizer)]
        inputs_ids = data_collator(inputs, tokenizer)
        outputs = model(**inputs_ids)
        answers = [extract_answer(inputs, outputs, tokenizer)]

    return answers

def answer_from_model(question, context):
    answer = '-1'
    results = get_answer(question, context)
    for result in results:
        answer = result[0]['answer']
    return answer

def answer_bm25semantic(question):
    question = clean_sem(question)
    arr = bm25_search_s(question,  limit=2)
    context1 = arr[0]
    context2 = arr[1]
    sub1 = three_sub_relevant(question, context1)
    sub2 = three_sub_relevant(question, context2)
    answer = ''
    context = ''
    for i in sub1:
        a =  answer_from_model(question, i)
        if a != '':
            answer = a
            context = i
            return answer, context
    for i in sub2:
        b = answer_from_model(question, i)
        if b!='':
            answer = b
            context = i
        return answer, context

    return answer

In [35]:
def qa_semantic(question):
    question = clean_sem(question)
    arr, doc_scores = bm25_search(question, limit=2, k1=1.5, b=0.75)
    context1 = arr[0]
    context2 = arr[1]
    sub1 = three_sub_relevant(question, context1)
    sub2 = three_sub_relevant(question, context2)
    answer = ''
    context = ''
    for i in sub1:
        a = answer_from_model(question, i)
        if a != '':
            answer = a
            context = i
            return answer, context, sub1
    for i in sub2:
        b = answer_from_model(question, i)
        if b != '':
            answer = b
            context = i
        return answer, context, sub2

    return answer

In [36]:
simCSE = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

No sentence-transformers model found with name C:\Users\ASUS/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\ASUS/.cache\torch\sentence_transformers\VoVanPhuc_sup-SimCSE-VietNamese-phobert-base were not used when initializing RobertaModel: ['mlp.dense.bias', 'mlp.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
qa_semantic('Hội nghị Ianta diễn ra khi nào?')

('từ ngày 4 đến ngày 11-2-1945',
 'giới sau chiến tranh 3 phân chia thành quả chiến thắng giữa các nước thắng trận trong bối cảnh đó một hội nghị quốc tế đã được triệu tập tại ianta từ ngày 4 đến ngày 11-2-1945 với sự tham dự của nguyên thủ ba cường quốc là xtalin ph rudoven và u sớcsin',
 ['rudoven và u sớcsin',
  'giới sau chiến tranh 3 phân chia thành quả chiến thắng giữa các nước thắng trận trong bối cảnh đó một hội nghị quốc tế đã được triệu tập tại ianta từ ngày 4 đến ngày 11-2-1945 với sự tham dự của nguyên thủ ba cường quốc là xtalin ph rudoven và u sớcsin',
  'c122 hội nghị ianta đầu năm 1945 chiến tranh thế giới thứ hai bước vào giai đoạn kết thúc nhiều vấn đề quan trọng và cấp bách đặt ra trước các cường quốc đồng minh đó là 1 nhanh chóng đánh bại hoàn toàn các nước phát xít 2 tổ chức lại thế giới sau chiến tranh'])

### Xây dựng metric cho MRC

In [35]:
import time

In [32]:
from tqdm import tqdm

In [37]:
def clean(text):
    text = text.lower()
    punc = '''!()[]{};:'\,"”“<>./?@#$%^&*_~'''
    for ele in text:
        if ele in punc:
            text = text.replace(ele, "")
    cleaned_text = ' '.join(text.strip().split())
    return cleaned_text


def f1_score(predict, answer):
    pre = clean(predict)
    ans = clean(answer)
    pre = set(pre.split())
    ans = set(ans.split())
    inter = pre.intersection(ans)
    TP = len(pre.intersection(ans))
    FP = len(pre - inter)
    FN = len(ans - inter)
    if TP == 0:
        return 0
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    return 2/((1/precision) + (1/recall))


def mean_f1_score(ref_list, pre_list):
    total = 0
    for ref, pre in zip(ref_list, pre_list):
        total += f1_score(ref, pre)
    mean = total / len(ref_list)
    return mean

In [36]:
from evaluate import load
exact_match_metric = load("exact_match")

### Kết quả của BM25 + Semantic search

In [33]:
pred_semantic = []

for ques in tqdm(list_question):
    pred_semantic.append(qa_semantic(ques)[0])

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [02:00<00:00,  1.50s/it]


In [39]:
mean_f1_score(list_answer, pred_semantic)

0.41922370606332865

In [41]:
ignore = [" năm", "nước ", " nước", "năm "]

In [40]:
exact_match_metric.compute(references=list_answer, predictions=pred_semantic, regexes_to_ignore=ignore, ignore_case=True, ignore_punctuation=True)

{'exact_match': 0.325}

### Kết quả BM25

In [42]:
pred_answer = []
for i in tqdm(range(len(list_question))):
    answer = qa(list_question[i])[0]
    pred_answer.append(answer)
pred_answer

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [07:54<00:00,  5.93s/it]


['73%',
 'B. CLINTƠN',
 'ĐẦU NĂM 1945',
 '50 NƯỚC',
 'IANTA',
 'QUÂN ĐỘI MĨ',
 '13 TRIỆU NGƯỜI',
 'NHẬT BẢN',
 '-1',
 '-1',
 'HƠN 23 TRIỆU CỬ TRI',
 '-1',
 '94',
 'QUÂN TA TIẾN VÀO TIẾP QUẢN HÀ NỘI',
 'MIỀN BẮC CÓ TRÊN 85% HỘ NÔNG DÂN VỚI 70% RUỘNG ĐẤT VÀO HỢP TÁC XÃ NÔNG NGHIỆP , HƠN 8% SỐ THỢ THỦ CÔNG VÀ 45% SỐ NGƯỜI BUÔN BÁN NHỎ VÀO HỢP TÁC XÃ',
 '-1',
 '-1',
 'VĨ TUYẾN 38',
 '-1',
 'BÀN MÔN ĐIẾM',
 '17 NƯỚC',
 '<S> HỘI NGHỊ BAN CHẤP HÀNH TRUNG ƯƠNG ĐẢNG CỘNG SẢN ĐÔNG DƯƠNG THÁNG 7/1936 CHỦ TRƯƠNG THÀNH LẬP MẶT TRẬN NÀO ?</S> C.12.103 . HỘI NGHỊ BAN CHẤP HÀNH TRUNG ƯƠNG ĐẢNG CỘNG SẢN ĐÔNG DƯƠNG THÁNG 7-1936 THÁNG 7-1936 , HỘI NGHỊ BAN CHẤP HÀNH TRUNG ƯƠNG ĐẢNG CỘNG SẢN ĐÔNG DƯƠNG , DO LÊ HỒNG PHONG CHỦ TRÌ , HỌP Ở THƯỢNG HẢI . HỘI NGHỊ DỰA TRÊN NGHỊ QUYẾT ĐẠI HỘI LẦN THỨ VII CỦA QUỐC TẾ CỘNG SẢN , CĂN CỨ VÀO TÌNH HÌNH CỤ THỂ CỦA VIỆT NAM ĐỂ ĐỊNH RA ĐƯỜNG LỐI VÀ PHƯƠNG PHÁP ĐẤU TRANH . HỘI NGHỊ XÁC ĐỊNH : NHIỆM VỤ CHIẾN LƯỢC CỦA CÁCH MẠNG TƯ SẢN DÂN QUYỀN ĐÔNG DƯƠNG LÀ CHỐNG ĐẾ QUỐC 

In [43]:
mean_f1_score(pred_answer, list_answer)

0.5537775699364573

In [44]:
results1 = exact_match_metric.compute(references=list_answer, predictions=pred_answer, regexes_to_ignore=ignore, ignore_case=True, ignore_punctuation=True)
print(round(results1["exact_match"], 2))

0.38
